In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
df = pd.read_csv('MaineAllBallots.csv')
df.head()

,Precinct,1st Choice,2nd Choice,3rd Choice,Count,count
0,MINNEAPOLIS W-3 P-02,Chris Meyer,undervote,undervote,1.0,NaN
1,MINNEAPOLIS W-3 P-05,Mohamed Issa Barre,Chris Meyer,Billy Menz,1.0,NaN
2,MINNEAPOLIS W-2 P-05,Billy Menz,undervote,undervote,1.0,NaN
3,MINNEAPOLIS W-1 P-04,Billy Menz,Chris Meyer,undervote,1.0,NaN
4,MINNEAPOLIS W-1 P-02,Billy Menz,Chris Meyer,Mohamed Issa Barre,1.0,NaN


In [24]:
df_clean = df[~df.eq("undervote").any(axis=1)]
df_clean = df[~df.eq("overvote").any(axis=1)]
df_clean = df[~df.eq("nan").any(axis=1)]

In [25]:
df_clean.to_csv('cleaned_MaineData.csv', index=False)

In [26]:
df_clean.head()

,Precinct,1st Choice,2nd Choice,3rd Choice,Count,count
0,MINNEAPOLIS W-3 P-05,Mohamed Issa Barre,Chris Meyer,Billy Menz,1.0,NaN
1,MINNEAPOLIS W-1 P-02,Billy Menz,Chris Meyer,Mohamed Issa Barre,1.0,NaN
2,MINNEAPOLIS W-3 P-05,Billy Menz,Chris Meyer,Mohamed Issa Barre,1.0,NaN
3,MINNEAPOLIS W-1 P-07,Chris Meyer,Billy Menz,Mohamed Issa Barre,1.0,NaN
4,MINNEAPOLIS W-3 P-03,Chris Meyer,Billy Menz,Mohamed Issa Barre,1.0,NaN


In [27]:
df = pd.read_csv('cleaned_MaineData.csv')

In [28]:
first_column = df.columns[0]

number_counts = df[first_column].value_counts()

print(f"Number of unique values in the first column: {len(number_counts)}")
print(number_counts)

Number of unique values in the first column: 132
Precinct
MINNEAPOLIS W-3 P-06     1443
MINNEAPOLIS W-11 P-04    1351
MINNEAPOLIS W-3 P-03     1343
MINNEAPOLIS W-8 P-07     1183
MINNEAPOLIS W-9 P-02     1163
                         ... 
MINNEAPOLIS W-13 P-12     164
MINNEAPOLIS W-2 P-04      146
MINNEAPOLIS W-5 P-6C      122
MINNEAPOLIS W-9 P-08       77
MINNEAPOLIS W-12 P-07      35
Name: count, Length: 132, dtype: int64


In [31]:
ballots_df = pd.read_csv('MaineBallotsClean.csv')


In [33]:

# Convert each row into a tuple: (preferences list, weight)
ballots = [(list(row[:-1]), row[-1]) for row in ballots_df.values.tolist()]

# Identify all unique candidates (from the ranked preferences)
candidates = set()
for ballot, weight in ballots:
    for candidate in ballot:
        candidates.add(candidate)
candidates = list(candidates)

def count_votes(ballots, candidates):
    """
    Count the votes for the current round using weighted ballots.
    Each ballot contributes its weight to the vote of the highest-ranked candidate
    that hasn't been eliminated.
    """
    vote_counts = {candidate: 0 for candidate in candidates}
    for ballot, weight in ballots:
        for candidate in ballot:
            if candidate in candidates:
                vote_counts[candidate] += weight
                break  # Only count the top remaining candidate.
    return vote_counts

round_num = 1
while True:
    # Tally the weighted votes for this round.
    vote_counts = count_votes(ballots, candidates)
    total_votes = sum(vote_counts.values())
    print(f"Round {round_num} vote counts: {vote_counts} (Total active votes: {total_votes})")

    # Check if any candidate has a majority (>50% of active votes)
    for candidate, count in vote_counts.items():
        if count > total_votes / 2:
            print(f"\nWinner: {candidate} with {count} votes (majority of {total_votes} active votes)!")
            winner = candidate
            break
    else:
        # No candidate has a majority; eliminate candidate(s) with the fewest votes.
        min_votes = min(vote_counts.values())
        eliminated = [candidate for candidate, count in vote_counts.items() if count == min_votes]
        print(f"Eliminated in round {round_num}: {eliminated}\n")
        
        # Remove the eliminated candidate(s) from further consideration.
        for candidate in eliminated:
            candidates.remove(candidate)
        
        # If no candidates remain or all remaining are tied, declare a tie.
        if not candidates or len(vote_counts) == len(eliminated):
            print("Election resulted in a tie among the remaining candidates:")
            print(vote_counts)
            break

        round_num += 1
        continue
    break

Round 1 vote counts: {'Abdi Warsame': 712, 'Gary Schiff': 936, 'Andrea Fahrenkrug': 1522, 'Joe Kovacs': 230, 'Blong Yang': 915, 'Bill Shroyer': 1515, 'Samantha Pree-Stinson': 688, 'Steffanie Musich': 3509, 'Jennifer Zielinski': 1128, 'Saralyn Romanishan': 978, 'Bob Fine': 2356, 'Steve Fletcher': 1553, 'Andrew Johnson': 2433, 'Tim Bildsoe': 1170, 'Jeremy Schroeder': 1738, 'Lisa Bender': 2531, 'April Kane': 428, 'Billy Menz': 2083, 'Charles Exner': 929, 'Tom Nordyke': 379, 'Harrison Bullard': 191, 'Lisa Goodman': 1933, 'Ak Hassan': 2652, 'Phillipe M. Cunningham': 1261, 'Andrea Jenkins': 2435, 'Stephanie Gasca': 406, 'Chris Meyer': 2765, 'Terry White': 566, 'Ronald W. Peterson': 81, 'Dana Hansen': 114, 'Mohamud Noor': 899, 'Erica Mauter': 1648, 'David Holsinger': 181, 'Teqen Zéa-Aida': 592, 'Kevin Reich': 1704, 'Will Jaeger': 431, 'Ginger Jentzen': 2006, 'Fadumo Yusuf': 73, 'Alondra Cano': 1400, 'Bruce Lundeen': 185, 'Uwi': 402, 'John Quincy': 1530, 'Mohamed Issa Barre': 1496, 'Tiffini Fo

In [34]:
ballots = [(list(row[:-1]), row[-1]) for row in ballots_df.values.tolist()]

# Identify all unique candidates from the ranked preferences.
original_candidates = set()
for ballot, weight in ballots:
    for candidate in ballot:
        original_candidates.add(candidate)
original_candidates = list(original_candidates)

def count_votes(ballots, candidates):
    """
    Count the weighted votes for the current round.
    For each ballot, the vote is assigned to the highest-ranked candidate
    that hasn't been eliminated.
    """
    vote_counts = {candidate: 0 for candidate in candidates}
    for ballot, weight in ballots:
        for candidate in ballot:
            if candidate in candidates:
                vote_counts[candidate] += weight
                break  # Only count the top remaining candidate.
    return vote_counts

def simulate_irv(ballots, original_candidates):
    """
    Simulate an IRV election on the given ballots.
    Returns the winning candidate (or None if there's a tie).
    """
    # Work on a copy of the candidate list.
    candidates = list(original_candidates)
    round_num = 1
    while True:
        vote_counts = count_votes(ballots, candidates)
        total_votes = sum(vote_counts.values())
        print(f"Round {round_num} vote counts: {vote_counts} (Total active votes: {total_votes})")
        
        # Check if any candidate has a majority (>50%).
        for candidate, count in vote_counts.items():
            if count > total_votes / 2:
                print(f"\nWinner: {candidate} with {count} votes (majority of {total_votes} active votes)!")
                return candidate
        
        # No majority found; eliminate candidate(s) with the fewest votes.
        min_votes = min(vote_counts.values())
        eliminated = [candidate for candidate, count in vote_counts.items() if count == min_votes]
        print(f"Eliminated in round {round_num}: {eliminated}\n")
        
        # Remove eliminated candidate(s) from further consideration.
        for candidate in eliminated:
            candidates.remove(candidate)
        
        # If no candidates remain or all remaining candidates are tied, declare a tie.
        if not candidates or len(vote_counts) == len(eliminated):
            print("Election resulted in a tie among the remaining candidates:")
            print(vote_counts)
            return None
        
        round_num += 1

# ------------------------------
# Run simulation with full ballots
# ------------------------------
print("Running IRV simulation with full ballots:")
winner_full = simulate_irv(ballots, original_candidates)

# ------------------------------
# Randomly eliminate an X percent of the ballots and simulate again.
# ------------------------------
elimination_percent = 0.1  # Change this value for a different percentage (e.g., 0.1 for 10%).
num_ballots = len(ballots)
num_to_remove = int(num_ballots * elimination_percent)

# Randomly select ballots to remove. The remaining ballots will be used for the simulation.
ballots_reduced = random.sample(ballots, num_ballots - num_to_remove)

print("\nRunning IRV simulation with reduced ballots (after randomly eliminating {:.0f}% of ballots):".format(elimination_percent * 100))
winner_reduced = simulate_irv(ballots_reduced, original_candidates)

# ------------------------------
# Compare the results.
# ------------------------------
print("\nComparison of Results:")
print("Winner with full ballots:", winner_full)
print("Winner with reduced ballots:", winner_reduced)

Running IRV simulation with full ballots:
Round 1 vote counts: {'Abdi Warsame': 712, 'Gary Schiff': 936, 'Andrea Fahrenkrug': 1522, 'Joe Kovacs': 230, 'Blong Yang': 915, 'Bill Shroyer': 1515, 'Samantha Pree-Stinson': 688, 'Steffanie Musich': 3509, 'Jennifer Zielinski': 1128, 'Saralyn Romanishan': 978, 'Bob Fine': 2356, 'Steve Fletcher': 1553, 'Andrew Johnson': 2433, 'Tim Bildsoe': 1170, 'Jeremy Schroeder': 1738, 'Lisa Bender': 2531, 'April Kane': 428, 'Billy Menz': 2083, 'Charles Exner': 929, 'Tom Nordyke': 379, 'Harrison Bullard': 191, 'Lisa Goodman': 1933, 'Ak Hassan': 2652, 'Phillipe M. Cunningham': 1261, 'Andrea Jenkins': 2435, 'Stephanie Gasca': 406, 'Chris Meyer': 2765, 'Terry White': 566, 'Ronald W. Peterson': 81, 'Dana Hansen': 114, 'Mohamud Noor': 899, 'Erica Mauter': 1648, 'David Holsinger': 181, 'Teqen Zéa-Aida': 592, 'Kevin Reich': 1704, 'Will Jaeger': 431, 'Ginger Jentzen': 2006, 'Fadumo Yusuf': 73, 'Alondra Cano': 1400, 'Bruce Lundeen': 185, 'Uwi': 402, 'John Quincy': 153

NameError: name 'random' is not defined

In [35]:
import pandas as pd
import random  # Make sure to import the random module

# Convert each row into a tuple: (preferences list, weight)
ballots = [(list(row[:-1]), row[-1]) for row in ballots_df.values.tolist()]

# Identify all unique candidates from the ranked preferences.
original_candidates = set()
for ballot, weight in ballots:
    for candidate in ballot:
        original_candidates.add(candidate)
original_candidates = list(original_candidates)

# ------------------------------
# Define helper functions for IRV simulation
# ------------------------------
def count_votes(ballots, candidates):
    """
    Count the weighted votes for the current round.
    Each ballot contributes its weight to the vote of the highest-ranked candidate
    that hasn't been eliminated.
    """
    vote_counts = {candidate: 0 for candidate in candidates}
    for ballot, weight in ballots:
        for candidate in ballot:
            if candidate in candidates:
                vote_counts[candidate] += weight
                break  # Only count the top remaining candidate.
    return vote_counts

def simulate_irv(ballots, original_candidates, verbose=False):
    """
    Simulate an IRV election on the given ballots.
    Returns the winning candidate (or None if there's a tie).
    
    If verbose is True, prints detailed round-by-round information.
    """
    # Work on a copy of the candidate list.
    candidates = list(original_candidates)
    round_num = 1
    while True:
        vote_counts = count_votes(ballots, candidates)
        total_votes = sum(vote_counts.values())
        if verbose:
            print(f"Round {round_num} vote counts: {vote_counts} (Total active votes: {total_votes})")
        
        # Check if any candidate has a majority (>50% of active votes)
        for candidate, count in vote_counts.items():
            if count > total_votes / 2:
                if verbose:
                    print(f"\nWinner: {candidate} with {count} votes (majority of {total_votes} active votes)!")
                return candidate
        
        # No majority found; eliminate candidate(s) with the fewest votes.
        min_votes = min(vote_counts.values())
        eliminated = [candidate for candidate, count in vote_counts.items() if count == min_votes]
        if verbose:
            print(f"Eliminated in round {round_num}: {eliminated}\n")
        
        # Remove the eliminated candidate(s) from further consideration.
        for candidate in eliminated:
            candidates.remove(candidate)
        
        # If no candidates remain or all remaining candidates are tied, declare a tie.
        if not candidates or len(vote_counts) == len(eliminated):
            if verbose:
                print("Election resulted in a tie among the remaining candidates:")
                print(vote_counts)
            return None
        
        round_num += 1

# ------------------------------
# Run multiple simulations and record the results
# ------------------------------
num_simulations = 1000  # Set this to the desired number of simulations.
elimination_percent = 0.9  # Percent (e.g., 0.1 for 10%) of ballots to eliminate for the reduced simulation.

results = []  # List to store each simulation's results.

for sim in range(num_simulations):
    print(f"\nRunning simulation {sim + 1}...")
    
    # Run simulation with full ballots.
    winner_full = simulate_irv(ballots, original_candidates, verbose=False)
    
    # For the reduced simulation, randomly eliminate the specified percent of ballots.
    num_ballots = len(ballots)
    num_to_remove = int(num_ballots * elimination_percent)
    ballots_reduced = random.sample(ballots, num_ballots - num_to_remove)
    winner_reduced = simulate_irv(ballots_reduced, original_candidates, verbose=False)
    
    # Record the winners.
    results.append({
        'simulation': sim + 1,
        'winner_full': winner_full,
        'winner_reduced': winner_reduced
    })

# ------------------------------
# Save the simulation results to a CSV file
# ------------------------------
results_df = pd.DataFrame(results)
results_df.to_csv('simulation_results_10.csv', index=False)
print("\nSimulation results saved to simulation_results_10.csv")


Running simulation 1...

Running simulation 2...

Running simulation 3...

Running simulation 4...

Running simulation 5...

Running simulation 6...

Running simulation 7...

Running simulation 8...

Running simulation 9...

Running simulation 10...

Running simulation 11...

Running simulation 12...

Running simulation 13...

Running simulation 14...

Running simulation 15...

Running simulation 16...

Running simulation 17...

Running simulation 18...

Running simulation 19...

Running simulation 20...

Running simulation 21...

Running simulation 22...

Running simulation 23...

Running simulation 24...

Running simulation 25...

Running simulation 26...

Running simulation 27...

Running simulation 28...

Running simulation 29...

Running simulation 30...

Running simulation 31...

Running simulation 32...

Running simulation 33...

Running simulation 34...

Running simulation 35...

Running simulation 36...

Running simulation 37...

Running simulation 38...

Running simulation 3

In [36]:
ballots = [(list(row[:3]), row[-1]) for row in ballots_df.values.tolist()]

# Identify all unique candidates from the first 3 ranked preferences.
original_candidates = set()
for ballot, weight in ballots:
    for candidate in ballot:
        original_candidates.add(candidate)
original_candidates = list(original_candidates)

# ------------------------------
# Define helper functions for IRV simulation
# ------------------------------
def count_votes(ballots, candidates):
    """
    Count the weighted votes for the current round.
    Each ballot contributes its weight to the vote of the highest-ranked candidate
    that hasn't been eliminated.
    """
    vote_counts = {candidate: 0 for candidate in candidates}
    for ballot, weight in ballots:
        for candidate in ballot:
            if candidate in candidates:
                vote_counts[candidate] += weight
                break  # Only count the top remaining candidate.
    return vote_counts

def simulate_irv(ballots, original_candidates):
    """
    Simulate an IRV election on the given ballots.
    Returns the winning candidate (or None if there's a tie).
    """
    # Work on a copy of the candidate list.
    candidates = list(original_candidates)
    round_num = 1
    while True:
        vote_counts = count_votes(ballots, candidates)
        total_votes = sum(vote_counts.values())
        print(f"Round {round_num} vote counts: {vote_counts} (Total active votes: {total_votes})")
        
        # Check if any candidate has a majority (>50% of active votes)
        for candidate, count in vote_counts.items():
            if count > total_votes / 2:
                print(f"\nWinner: {candidate} with {count} votes (majority of {total_votes} active votes)!")
                return candidate
        
        # No majority found; eliminate candidate(s) with the fewest votes.
        min_votes = min(vote_counts.values())
        eliminated = [candidate for candidate, count in vote_counts.items() if count == min_votes]
        print(f"Eliminated in round {round_num}: {eliminated}\n")
        
        # Remove the eliminated candidate(s) from further consideration.
        for candidate in eliminated:
            candidates.remove(candidate)
        
        # If no candidates remain or all remaining candidates are tied, declare a tie.
        if not candidates or len(vote_counts) == len(eliminated):
            print("Election resulted in a tie among the remaining candidates:")
            print(vote_counts)
            return None
        
        round_num += 1

# ------------------------------
# Run simulation with truncated ballots
# ------------------------------
print("Running IRV simulation with truncated ballots (top 3 preferences only):")
winner = simulate_irv(ballots, original_candidates)
print("\nWinner with truncated ballots:", winner)

Running IRV simulation with truncated ballots (top 3 preferences only):
Round 1 vote counts: {'Abdi Warsame': 712, 'Gary Schiff': 936, 'Andrea Fahrenkrug': 1522, 'Joe Kovacs': 230, 'Blong Yang': 915, 'Bill Shroyer': 1515, 'Samantha Pree-Stinson': 688, 'Steffanie Musich': 3509, 'Jennifer Zielinski': 1128, 'Saralyn Romanishan': 978, 'Bob Fine': 2356, 'Steve Fletcher': 1553, 'Andrew Johnson': 2433, 'Tim Bildsoe': 1170, 'Jeremy Schroeder': 1738, 'Lisa Bender': 2531, 'April Kane': 428, 'Billy Menz': 2083, 'Charles Exner': 929, 'Tom Nordyke': 379, 'Harrison Bullard': 191, 'Lisa Goodman': 1933, 'Ak Hassan': 2652, 'Phillipe M. Cunningham': 1261, 'Andrea Jenkins': 2435, 'Stephanie Gasca': 406, 'Chris Meyer': 2765, 'Terry White': 566, 'Ronald W. Peterson': 81, 'Dana Hansen': 114, 'Mohamud Noor': 899, 'Erica Mauter': 1648, 'David Holsinger': 181, 'Teqen Zéa-Aida': 592, 'Kevin Reich': 1704, 'Will Jaeger': 431, 'Ginger Jentzen': 2006, 'Fadumo Yusuf': 73, 'Alondra Cano': 1400, 'Bruce Lundeen': 185, 

In [37]:
import pandas as pd
import random

# ------------------------------
# Set the number of ranked candidates to consider (Change this value as needed)
# ------------------------------
num_ranked_candidates = 5  # Change this value to the desired number of ranked candidates

# ------------------------------
# Load and prepare the ballots data
# ------------------------------
ballots_df = pd.read_csv('exact_top5_ballots.csv')  # Replace with the actual file path

# Convert each row into a tuple: (limited candidate preferences, weight)
ballots = [(list(row[:num_ranked_candidates]), row[-1]) for row in ballots_df.values.tolist()]

# Identify all unique candidates from the selected number of ranked preferences
original_candidates = set()
for ballot, weight in ballots:
    for candidate in ballot:
        original_candidates.add(candidate)
original_candidates = list(original_candidates)

# ------------------------------
# Define helper functions for IRV simulation
# ------------------------------
def count_votes(ballots, candidates):
    """
    Count the weighted votes for the current round.
    Each ballot contributes its weight to the vote of the highest-ranked candidate
    that hasn't been eliminated.
    """
    vote_counts = {candidate: 0 for candidate in candidates}
    for ballot, weight in ballots:
        for candidate in ballot:
            if candidate in candidates:
                vote_counts[candidate] += weight
                break  # Only count the top remaining candidate.
    return vote_counts

def simulate_irv(ballots, original_candidates):
    """
    Simulate an IRV election on the given ballots.
    Returns the winning candidate (or None if there's a tie).
    """
    # Work on a copy of the candidate list.
    candidates = list(original_candidates)
    round_num = 1
    while True:
        vote_counts = count_votes(ballots, candidates)
        total_votes = sum(vote_counts.values())
        print(f"Round {round_num} vote counts: {vote_counts} (Total active votes: {total_votes})")
        
        # Check if any candidate has a majority (>50% of active votes)
        for candidate, count in vote_counts.items():
            if count > total_votes / 2:
                print(f"\nWinner: {candidate} with {count} votes (majority of {total_votes} active votes)!")
                return candidate
        
        # No majority found; eliminate candidate(s) with the fewest votes.
        min_votes = min(vote_counts.values())
        eliminated = [candidate for candidate, count in vote_counts.items() if count == min_votes]
        print(f"Eliminated in round {round_num}: {eliminated}\n")
        
        # Remove the eliminated candidate(s) from further consideration.
        for candidate in eliminated:
            candidates.remove(candidate)
        
        # If no candidates remain or all remaining candidates are tied, declare a tie.
        if not candidates or len(vote_counts) == len(eliminated):
            print("Election resulted in a tie among the remaining candidates:")
            print(vote_counts)
            return None
        
        round_num += 1

# ------------------------------
# Run simulation with predefined ranked candidates
# ------------------------------
print(f"Running IRV simulation with top {num_ranked_candidates} preferences only:")
winner = simulate_irv(ballots, original_candidates)
print(f"\nWinner with top {num_ranked_candidates} ranked candidates:", winner)

Running IRV simulation with top 5 preferences only:
Round 1 vote counts: {217796: 1311, 217572: 2962, 219978: 1981, 219469: 2842, 218491: 477} (Total active votes: 9573)
Eliminated in round 1: [218491]

Round 2 vote counts: {217796: 1360, 217572: 3059, 219978: 2111, 219469: 3043} (Total active votes: 9573)
Eliminated in round 2: [217796]

Round 3 vote counts: {217572: 3518, 219978: 2638, 219469: 3417} (Total active votes: 9573)
Eliminated in round 3: [219978]

Round 4 vote counts: {217572: 4589, 219469: 4984} (Total active votes: 9573)

Winner: 219469 with 4984 votes (majority of 9573 active votes)!

Winner with top 5 ranked candidates: 219469


In [1]:
import pandas as pd

# Define the top 5 candidates as a set (adjust these values if needed)
top5_candidates = {"217572", "219469", "219978", "217796", "218491"}

# Load the cleaned Mayor data CSV.
# It is assumed that each row has candidate rankings in the first columns and the last column is the ballot weight.
ballots_df = pd.read_csv('cleaned_MayorData.csv')

# Create a list of tuples: (preferences, weight)
ballots = [(list(row[:-1]), row[-1]) for row in ballots_df.values.tolist()]

# Filter to keep only ballots that contain exactly the top 5 candidates (no more, no less)
exact_top5_ballots = [ (prefs, weight) for (prefs, weight) in ballots if set(prefs) == top5_candidates ]

print("Number of ballots with exactly the top 5 candidates:", len(exact_top5_ballots))

# Prepare data for output.
# Assuming each ballot in the CSV originally has 5 candidate columns plus a weight column,
# we can create a new DataFrame with columns for each candidate ranking and the weight.
data = []
for prefs, weight in exact_top5_ballots:
    # Here, we assume the ballot contains exactly 5 preferences.
    row = {
        "Candidate_1": prefs[0],
        "Candidate_2": prefs[1],
        "Candidate_3": prefs[2],
        "Candidate_4": prefs[3],
        "Candidate_5": prefs[4],
        "Weight": weight
    }
    data.append(row)

df_exact_top5 = pd.DataFrame(data)

# Save the filtered ballots to a new CSV file.
output_file = "exact_top5_ballots.csv"
df_exact_top5.to_csv(output_file, index=False)
print(f"Filtered ballots saved to {output_file}")


Number of ballots with exactly the top 5 candidates: 120
Filtered ballots saved to exact_top5_ballots.csv
